<a href="https://colab.research.google.com/github/maverick2903/SEM6_Programs/blob/main/ML/CART_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np

class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # For leaf nodes, the class label

class CARTDecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2, min_impurity=1e-7):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity = min_impurity

    def fit(self, X, y):
        self.num_classes = len(np.unique(y))
        self.num_features = X.shape[1]
        self.tree = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        num_samples_per_class = [np.sum(y == i) for i in range(self.num_classes)]
        predicted_class = np.argmax(num_samples_per_class)
        node = Node(value=predicted_class)

        if depth < self.max_depth:
            best_split = None
            best_gini = float('inf')
            impurity_node = self._gini_impurity(y)

            for feature_index in range(self.num_features):
                unique_values = np.unique(X[:, feature_index])
                for threshold in unique_values:
                    left_indices = np.where(X[:, feature_index] <= threshold)[0]
                    right_indices = np.where(X[:, feature_index] > threshold)[0]

                    if len(left_indices) < self.min_samples_split or len(right_indices) < self.min_samples_split:
                        continue

                    y_left = y[left_indices]
                    y_right = y[right_indices]

                    gini = (len(y_left) / len(y)) * self._gini_impurity(y_left) + \
                           (len(y_right) / len(y)) * self._gini_impurity(y_right)

                    if gini < best_gini:
                        best_split = {
                            'feature_index': feature_index,
                            'threshold': threshold,
                            'left_indices': left_indices,
                            'right_indices': right_indices
                        }
                        best_gini = gini

            if best_gini < impurity_node:
                left = self._grow_tree(X[best_split['left_indices']], y[best_split['left_indices']], depth + 1)
                right = self._grow_tree(X[best_split['right_indices']], y[best_split['right_indices']], depth + 1)
                node = Node(feature_index=best_split['feature_index'], threshold=best_split['threshold'], left=left, right=right)

        return node

    def _gini_impurity(self, y):
        hist = np.bincount(y)
        probs = hist / len(y)
        return 1 - np.sum(probs ** 2)

    def predict(self, X):
        return np.array([self._predict_tree(x, self.tree) for x in X])

    def _predict_tree(self, x, tree):
        if tree.value is not None:
            return tree.value
        feature_value = x[tree.feature_index]
        if feature_value <= tree.threshold:
            return self._predict_tree(x, tree.left)
        else:
            return self._predict_tree(x, tree.right)

    def _show_tree(self, node, depth=0, cond=""):
        base = '    ' * depth + cond
        if node.feature_index is not None:
            print(base + 'if X[' + str(node.feature_index) + '] <= ' + str(node.threshold))
            self._show_tree(node.left, depth + 1, 'then ')
            self._show_tree(node.right, depth + 1, 'else ')
        else:
            print(base + '{value: ' + str(node.value) + '}')

    def print_tree(self):
        self._show_tree(self.tree)

def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)


data = load_breast_cancer()
X = data.data
y = data.target


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the decision tree
tree = CARTDecisionTree(max_depth=5)
tree.fit(X_train, y_train)

# Predict
y_pred_train = tree.predict(X_train)
y_pred_test = tree.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy(y_train, y_pred_train)
test_accuracy = accuracy(y_test, y_pred_test)

print(f"Training accuracy: {train_accuracy:.4f}")
print(f"Testing accuracy: {test_accuracy:.4f}")

# Print the tree rules
print("Tree Rules:")
tree.print_tree()


Training accuracy: 0.9912
Testing accuracy: 0.9298
Tree Rules:
if X[7] <= 0.05074
    then if X[20] <= 16.77
        then if X[10] <= 0.6061
            then if X[24] <= 0.1733
                then if X[14] <= 0.00328
                    then {value: 1}
                    else {value: 1}
                else {value: 0}
            else {value: 0}
        else if X[1] <= 15.7
            then {value: 1}
            else if X[17] <= 0.009921
                then {value: 0}
                else {value: 1}
    else if X[27] <= 0.1465
        then if X[22] <= 114.3
            then if X[1] <= 20.76
                then {value: 1}
                else {value: 0}
            else {value: 0}
        else if X[16] <= 0.1278
            then {value: 0}
            else {value: 1}
